In [1]:
import os
from openai import OpenAI
import json
import collections

import subprocess
import sys


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

from typing import Literal


In [4]:
# core vairables to import from src 
from src import models, TEMP
# main functions to import from src
from src import GGB_Statements, Single_Agent_Handler, get_prompt, get_client

# helper functions to import from src to handle filenaming and checkpooints
from src import extract_confidence_from_response_single, extract_answer_from_response_single, get_consistent_filenames, save_checkpoint, load_checkpoint

In [5]:
QUESTION_JSON = os.path.abspath('GGB_benchmark/OUS.json') 
Inverted_JSON = os.path.abspath('GGB_benchmark/OUSinverted.json') 
Qs = GGB_Statements(QUESTION_JSON) 
iQs = GGB_Statements(Inverted_JSON)
